In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics, preprocessing, linear_model
np.random.seed(0)



In [2]:
print("# Loading data...")
# The training data is used to train your model how to predict the targets.
data = pd.read_csv('training_data.csv', header=0)
data = data.drop([
    'id', 'era', 'data_type', 'target_charles', 'target_elizabeth',
    'target_jordan', 'target_ken', 'target_frank', 'target_hillary'],axis=1)

# Transform the loaded CSV data into numpy arrays
features = [f for f in list(data) if "feature" in f]
X = data[features]
Y = data['target_bernie']



# Loading data...


In [4]:
# This is your model that will learn to predict this target.
model = linear_model.LogisticRegression(n_jobs=-1)
print("# Training...")
# Your model is trained on train_bernie
model.fit(X, Y)

print("# Predicting...")
# Based on the model we can predict the probability of each row being
# a bernie_target in the validation data.
# The model returns two columns: [probability of 0, probability of 1]
# We are just interested in the probability that the target is 1.
y_prediction = model.predict_proba(Y)



# Training...


/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


# Predicting...


ValueError: Expected 2D array, got 1D array instead:
array=[1 1 0 ... 0 0 0].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
# We can see the probability does seem to be good at predicting the
# true target correctly.
print("- target:", validation['target_bernie'][1:6])
print("- rounded probability:", [round(p) for p in probabilities][1:6])

# But overall the accuracy is very low.
correct = [
round(x) == y
for (x, y) in zip(probabilities, validation['target_bernie'])
]
print("- accuracy: ", sum(correct) / float(validation.shape[0]))

# The targets for each of the tournaments are very correlated.
tournament_corr = np.corrcoef(validation['target_bernie'],
  validation['target_elizabeth'])
print("- bernie vs elizabeth corr:", tournament_corr)
# You can see that target_elizabeth is accurate using the bernie model as well.
correct = [
round(x) == y
for (x, y) in zip(probabilities, validation['target_elizabeth'])
]
print("- elizabeth using bernie:",
  sum(correct) / float(validation.shape[0]))

# Numerai measures models on logloss instead of accuracy. The lower the logloss the better.
# Numerai only pays models with logloss < 0.693 on the live portion of the tournament data.
# Our validation logloss isn't very good.
print("- validation logloss:",
  metrics.log_loss(validation['target_bernie'], probabilities))

# To submit predictions from your model to Numerai, predict on the entire tournament data.
x_prediction = tournament[features]
y_prediction = model.predict_proba(x_prediction)
results = y_prediction[:, 1]

print("# Creating submission...")
# Create your submission
results_df = pd.DataFrame(data={'probability': results})
joined = pd.DataFrame(ids).join(results_df)
print("- joined:", joined.head())

print("# Writing predictions to bernie_submissions.csv...")
# Save the predictions out to a CSV file.
joined.to_csv("bernie_submission.csv", index=False)